# C++ Jezgro


Po pokretanju vec je automatski izvrsena celija:
    #include <iostream>
    using namespace std;
    
%r je line magic i izvrsava kod kao sto bi bio izvrsen u main funkciji  
(Idealno  %r bi trebalo da bude statment magic u stilu c++ jezika i da podrzava vise linija)

%%r je cell magic i celu celiju tretira kao da je main funkcija

In [1]:
int a = 2;
%r cout << a << endl;

string b  = "Hello";
%r cout << b << endl;

// samo deklarisemo funkciju
void f(int i, const string & s){
    while(i--)
        cout << i << ":" << s << endl;
}



2
Hello


In [2]:
// deklarisemo jos jednu funkciju
void g(void){
    f(a, b);
}

In [3]:
%%r 
g();    
cout << endl;
a = 5;                                                                       
f(a, "World");  

1:Hello
0:Hello

4:World
3:World
2:World
1:World
0:World


Ukljucicemmo %debug da bi demonstrirali iza scene koji kod se zapravo izvrsava

In [4]:
%debug
%r cout << "Hy" << endl;
void p (int c){
    cout << c << " " <<  b << endl;
}
%r p(6);

%debug = True
<----code---->
#include <iostream>
using namespace std;

extern int a ;
extern string b  ;
void g(void);
void f(int i, const string & s);


void p (int c){
    cout << c << " " <<  b << endl;
}


void __run__(void) { 
  try{
       cout << "Hy" << endl;
 p(6);
      cout.flush();
  } catch (std::exception& e) {
      std::cout << "Exception catched : " << e.what() << std::endl;
  }
}
<--------->

Hy
6 Hello


In [5]:
// ne treba nam vise
%debug 0 

%debug = False


# Problem je redefinisanje simbola
Linker pretrazuje prvo .so koji ucitava pa onda krece od pocetka  
da pretrazuje sve ucitane .so (a trebalo bi od kraja, tako da nadje one najsvezije prvo)

In [6]:
long a = 3333;
%r cout << a << endl;

3333


Uveli smo novu definiciju long $a$ i to izgleda kao da radi medjutim...  
Ako ponovo pokusamo da pristupimo $a$ u drugoj celiji linker ce naci ono prvo int $a$... :(

In [7]:
%%r 
cout << a; // nasao je staro a :(
a = 42; // ali mozemo da promenimo vrednost

5

Ali mozemo da menjamo vrednost $a$ bez problema.


In [8]:
%r cout << a;

42

Ovo je ozbiljan problem jer nas sprecava da redefinisemo funkcije.

Jedno resenje bi bilo da se imena simbola numerisu i da se prati njihova numeracija.
Naime ako postoji simbol $a$ onda bi se u celiji koja redefinise $a$ svuda promenilo
ime simbola $a$ dodavanjem nekog prefiksa. Svaki sledeci put pri  kompilaciji sve pojave
simbola $a$ bi bile zamenjene najsvezijom verzijom sa prefiksom.


Ovo resenje zahteva ozbiljno  parsiranje (koriscenje libclang) i trenutno je out of the scope.


__ Medjutim nije ni to dovoljno  __

Ako bi redefinsalii funkciju $f$ to ne bi uticalo na njeno pozivanje u funkciji $g$.
Jer $g$ je vec definisano i simbol $f$ je resolv-ovan na staru funkciju. Tu se ne moze nista osim
da se redefinise $g$. (Ali onda mozda opet mora da se redefinise neki kod koji poziva $g$)

U ovom slucaju ima smisla da se definicija za $g$ automatski ponovo osvezi.  
Ali sta ako bi to osvezavanje napravilo bug?  
Da li je to korektno ponasanje za C++?

Da li mozda prosto koristiti pokazivace na funkcije a redefinisanje posmatrati kao dodelu novih vrednosti?
Jedino promena potipisa bi zahtevala novo definisanje pokazivaca na funkciju.

Type Erasure mozda ?

## Primeri greske za magic, kompilacije i linkinga

In [9]:
%run

%run is not defined

In [10]:
int ;

error: declaration does not declare anything [-fpermissive]
 int ;
 ^


In [11]:
extern int c;

%r a = c;

undefined symbol: c

## Jos neki nedostaci:
1. Segmentation fault nazalost ubija jezgro :(
2. Trenutno nije moguc interkativni ulaz i izlaz
3. Auto completition on the way (zahteva libclang)
4. Podrska za clase, strukture i sablone fali (ceka se libclang)